In [1]:
import solution_file_processing as sfp
import os
import numpy as np
import pandas as pd
from dask import dataframe as dd

from solution_file_processing.utils.utils import catch_errors
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE, IEA_PALETTE_16, IEA_PALETTE_PLUS, EXTENDED_PALETTE
from solution_file_processing.constants import VRE_TECHS, PRETTY_MODEL_NAMES
from solution_file_processing.timeseries import create_output_11 as create_ts_output_11
from solution_file_processing.timeseries import create_output_4 as create_timeseries_output_4
from solution_file_processing import log
from solution_file_processing.plots import _get_plot_1_variables

# Initialize confi||g with toml file
c = sfp.SolutionFilesConfig('config_files/ukraine/UKR.toml')

# c = sfp.SolutionFilesConfig('config_files/thailand/THA.toml')
from solution_file_processing.plots import create_plot_1a, create_plot_2b_ref_plots, create_plot_2_summary
from solution_file_processing.plots import create_plot_6_ldc_and_line_plots, create_plot_7_co2_savings, create_plot_8_services, create_plot_9_av_cap
from solution_file_processing.plots import create_plot_1a_overall_models

[12:05:12 00:00] I:121:__init__ - Logging to C:\Users\hart_c\showcase\solution-file-processing\logs\20241216_120512-UKR.log.
[12:05:14 00:01] I:166:__init__ - Initialized SolutionFilesConfig for config_files/ukraine/UKR.toml.


In [2]:
# price_all_regs_ts = c.o.reg_raw_df.compute()
# x = price_all_regs_ts.groupby(['model','name','timestamp']).agg({'value':'sum'})

In [7]:
c.v.export_cost_ts

value
model           timestamp                        
2021            2030-01-01 00:00:00 -1.421085e-09
                2030-01-01 01:00:00  0.000000e+00
                2030-01-01 02:00:00  0.000000e+00
                2030-01-01 03:00:00  2.842171e-09
                2030-01-01 04:00:00  0.000000e+00
...                                           ...
2030-BR_cPN_eSG 2030-12-31 19:00:00  4.999250e+06
                2030-12-31 20:00:00  4.999250e+06
                2030-12-31 21:00:00  4.999250e+06
                2030-12-31 22:00:00  3.784583e+06
                2030-12-31 23:00:00  3.784583e+06

[87600 rows x 1 columns]

In [13]:
export_cost_monthly = c.v.export_cost_ts.groupby([pd.Grouper(level='timestamp', freq='M'), 'model']).sum().value.unstack('model')

In [15]:
export_cost_monthly.columns

Index(['2021', '2024', '2025', '2025_eG', '2025_eSBaG', '2025_eSG',
       '2025_eSWBaG', '2025_eSWBaG_Ix', '2025_eSoG', '2030-BR_cPN_eSG'],
      dtype='object', name='model')

In [30]:
export_monthly = c.v.exports_ts.where(c.v.exports_ts.value > 0).groupby([pd.Grouper(level='timestamp', freq='M'), 'model']).sum().value.unstack('model')
import_monthly = -c.v.exports_ts.where(c.v.exports_ts.value < 0).groupby([pd.Grouper(level='timestamp', freq='M'), 'model']).sum().value.unstack('model')
net_export_monthly = c.v.exports_ts.groupby([pd.Grouper(level='timestamp', freq='M'), 'model']).sum().value.unstack('model')

In [39]:
c.v.exports_ts.groupby('model').sum()

,value
model,
2021,-1.381295e-11
2024,-1.449952e+07
2025,-1.622478e+07
2025_eG,-1.437512e+07
2025_eSBaG,-1.326134e+07
2025_eSG,-1.332416e+07
2025_eSWBaG,-1.275041e+07
2025_eSWBaG_Ix,-1.304380e+07
2025_eSoG,-1.385558e+07


In [3]:
c.p.import_cost[0].to_csv('y:/Modelling/Ukraine/2023_UKR_ST_Security/import_cost1.csv')

[13:41:37 00:00] I:52:_drive_cache_wrapper - Loading from variables cache: exports_ts.parquet (pd.DataFrame).
[13:41:37 00:00] I:49:_drive_cache_wrapper - Loading from objects cache: reg_raw_df.parquet (dd.DataFrame).


In [ ]:
from solution-file-processing.utils.utils import _prettify_model_name

In [ ]:
c.v.exports_ts.value

In [ ]:
c.p.import_cost

In [ ]:
price_by_reg_ts = c.o.reg_df[c.o.reg_df.property == 'Price'].groupby(['model', 'timestamp', c.GEO_COLS[0]]).agg({'value':'sum'}).compute()
price_by_reg_ts = price_by_reg_ts.unstack(c.GEO_COLS[0]).mean(axis=1)
export_cost_ts = c.v.exports_ts.value * price_by_reg_ts
export_cost_ts.groupby('model').sum()

In [ ]:
df_costs = c.v.export_cost_ts[c.v.export_cost_ts.value < 0].groupby('model').sum()/1e6
df_revenues = c.v.export_cost_ts[c.v.export_cost_ts.value > 0].groupby('model').sum()/1e6

In [ ]:

df = pd.concat([df_costs.value.rename('Costs'), df_revenues.value.rename('Revenues')], axis=1)
df

In [ ]:
export_cost_ts = c.v.exports_ts.value * price_by_reg_ts

In [ ]:
imp_exp_cost = c.v.exports_ts * price_by_reg

In [ ]:
c.o.reg_df[c.o.reg_df.property == 'Price'].compute()

In [ ]:
price_by_reg = price_by_reg.unstack(c.GEO_COLS[0])
price_by_reg = price_by_reg.mean(axis=1)